In [18]:
#! /usr/local/Cellar/jupyterlab/2.2.8/libexec/bin/python3.8 -m pip install networkx
import sklearn.metrics
import networkx as nx
from glob import glob
import igraph as ig
import numpy as np
import pandas as pd
import re
import os

In [10]:
def read_graph(path):
    edges = []
    labels = []
    with open(path, 'r') as f:
        next(f)
        next_line = next(f)
        if next_line.startswith('*'):
            for line in f:
                i, j, *_ = re.split('\s+', line.strip())
                i, j = int(i), int(j)
                edge = (i, j)
                edges.append(edge)
        else:
            labels.append(re.split('\s+', next_line.strip())[1])
            for line in f:
                if line.startswith('*'):
                    break
                labels.append(re.split('\s+', line.strip())[1])
            for line in f:
                i, j, *_ = re.split('\s+', line.strip())
                i, j = int(i), int(j)
                edge = (i, j)
                edges.append(edge)
    if labels:
        vertex_attrs = {'label': labels}
    else:
        vertex_attrs = {}
    g = ig.Graph(edges=edges, directed=False, vertex_attrs=vertex_attrs)
    g.vs.select(_degree=0).delete()
    g = g.simplify()
    return g

In [53]:
# Make dirs for plots
for algorithm in ['Leiden', 'Multi-level', 'Fast-greedy']:
    if not os.path.exists(algorithm):
        os.makedirs(algorithm)

In [60]:
#ig.plot(communities, mark_groups=True, layout=layout)

networks = []
leiden_mod = []
multilevel_mod = []
fastgreedy_mod = [] 


for path in glob('../real/*.net'):
    name = os.path.basename(path)
    networks.append(name) 
    g = read_graph(path)
    
    #LEIDEN
    leiden = g.community_leiden('modularity', n_iterations=-1)
    leiden_mod.append(leiden.modularity)
    
    leiden_plot = ig.plot(leiden, bbox = (300, 300), margin = 20)
    leiden_plot.save('Leiden/' + name.replace('.', '_') + ".png")
    
    #MULTI-LEVEL
    multilevel = g.community_multilevel()
    multilevel_mod.append(multilevel.modularity)
    
    ml_plot = ig.plot(multilevel, bbox = (300, 300), margin = 20)
    ml_plot.save('Multi-level/' + name.replace('.', '_') + ".png")
    
    #FAST GREEDY
    fastgreedy = g.community_fastgreedy().as_clustering()
    fastgreedy_mod.append(fastgreedy.modularity)
    fg = g.community_fastgreedy()
    
    fg_plot = ig.plot(fastgreedy, bbox = (300, 300), margin = 20)
    fg_plot.save('Fast-greedy/' + name.replace('.', '_') + ".png")


#SAVE TO TABLE
mod = {'Leiden': leiden_mod,
        'Multi-level': multilevel_mod,
        'Fast-greedy': fastgreedy_mod
        }
df = pd.DataFrame(mod, columns = ['Leiden','Multi-level', 'Fast-greedy'], index=networks)
print("MODULARITY TABLE\n", df)



MODULARITY TABLE
                           Leiden  Multi-level  Fast-greedy
emailEU.net             0.416260     0.402844     0.341373
cat_cortex_sim.net      0.266097     0.266097     0.260436
lastfm.net              0.818160     0.813778     0.796605
dolphins.net            0.527610     0.523338     0.495491
airports_UW.net         0.709798     0.691898     0.662490
PGP.net                 0.886520     0.882340     0.852488
emailURV.net            0.581637     0.542581     0.507024
zachary.net             0.419790     0.418803     0.380671
facebook.net            0.835705     0.834786     0.777381
football.net            0.604570     0.604570     0.549741
jazz.net                0.445144     0.443080     0.438908
celegans_metabolic.net  0.451453     0.440695     0.405497


In [8]:
g = ig.load('../real/zachary.net')
layout = g.layout_auto()